<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/PJH/Ensemble_(bagging).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
!pip install datasets


In [ ]:
import pandas as pd
import numpy as np
import re
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [ ]:
# 데이터 파일 로드
df = pd.read_csv('/content/drive/MyDrive/Labels_Safe_MSR+Juliet_random.csv')
df = df.drop(columns=["vul", "Unnamed: 0"])
df['code'] = df['code'].apply(lambda x: re.sub(r'/\*.*?\*/', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'//.*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include <.*?>\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include \".*?\"\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#define .*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'[\t ]+', ' ', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'\n\s*\n', '\n', x))

In [ ]:
# 데이터셋 생성
dataset = DatasetDict({
    'train': load_dataset('csv', data_files="preprocessed_data.csv")['train']
})

model_names = [
    "microsoft/graphcodebert-base",
    "microsoft/codebert-base-mlm",
    "microsoft/unixcoder-base"
]

models = [AutoModelForSequenceClassification.from_pretrained(name, num_labels=11) for name in model_names]
tokenizers = [AutoTokenizer.from_pretrained(name) for name in model_names]

batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/unixcoder-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out

In [ ]:
# 메트릭 계산 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"f1": f1_score(labels, predictions, average='weighted')}

In [ ]:
# K-Fold 교차 검증 설정
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [ ]:
for model, tokenizer in zip(models, tokenizers):
    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset['train'])):
        train_data = dataset['train'].select(train_idx)
        val_data = dataset['train'].select(val_idx)

        tokenized_train_data = train_data.map(lambda x: tokenizer(x['code'], padding="max_length", truncation=True, max_length=512), batched=True)
        tokenized_val_data = val_data.map(lambda x: tokenizer(x['code'], padding="max_length", truncation=True, max_length=512), batched=True)

        training_args = TrainingArguments(
            output_dir=f'./results/{model.__class__.__name__}',
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=64,
            num_train_epochs=5,
            weight_decay=0.01,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            greater_is_better=True
        )


Map:   0%|          | 0/13120 [00:00<?, ? examples/s]

Map:   0%|          | 0/3281 [00:00<?, ? examples/s]

Map:   0%|          | 0/13121 [00:00<?, ? examples/s]

Map:   0%|          | 0/3280 [00:00<?, ? examples/s]

Map:   0%|          | 0/13121 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3280 [00:00<?, ? examples/s]

Map:   0%|          | 0/13121 [00:00<?, ? examples/s]

Map:   0%|          | 0/3280 [00:00<?, ? examples/s]

Map:   0%|          | 0/13121 [00:00<?, ? examples/s]

Map:   0%|          | 0/3280 [00:00<?, ? examples/s]

In [ ]:
 early_stopping_callback = EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.01
        )

In [ ]:
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_train_data,
            eval_dataset=tokenized_val_data,
            compute_metrics=compute_metrics,
            callbacks=[early_stopping_callback]
        )
        trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.907700,0.477789,0.827560
2,0.428800,0.393599,0.862734


KeyboardInterrupt: 